In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
import pandas as pd
# from facenet_pytorch import InceptionResnetV1, fixed_image_standardization, training
# a = []
# while(1):
#     a.append(1)

In [2]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%%time
zip_path = '/content/drive/My Drive/Colab Notebooks/allimgs_face2.zip'
!cp "{zip_path}" .
!unzip -q allimgs_face2.zip 
!rm allimgs_face2.zip

CPU times: user 39 ms, sys: 17.1 ms, total: 56.1 ms
Wall time: 14.2 s


In [4]:
# # Get train and test data for classification
train_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_on_cand.csv')
new_paths = []
for path, winner in zip(train_df[train_df.columns[2]].tolist(), train_df[train_df.columns[3]].tolist()):
    words = path.split('/')
    new_path = 'allimgs_face/'+words[1]+'/'+words[2]
    if '.webp' not in new_path:
        new_paths.append([new_path, winner])
train_df = pd.DataFrame(new_paths) 
train_df[train_df.columns[1]]= train_df[train_df.columns[1]].astype(float)
# train_df = train_df.sample(frac=1).reset_index(drop=True)


test_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test_on_cand.csv')
new_paths_test = []
for path, winner in zip(test_df[test_df.columns[2]].tolist(), test_df[test_df.columns[3]].tolist()):
    words = path.split('/')
    new_path = 'allimgs_face/'+words[1]+'/'+words[2]
    if '.webp' not in new_path:
        new_paths_test.append([new_path, winner])

test_df = pd.DataFrame(new_paths_test)    
test_df[test_df.columns[1]]= test_df[test_df.columns[1]].astype(float)   

val_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/val_on_cand.csv')
new_paths_val = []
for path, winner in zip(val_df[val_df.columns[2]].tolist(), val_df[val_df.columns[3]].tolist()):
    words = path.split('/')
    new_path = 'allimgs_face/'+words[1]+'/'+words[2]
    if '.webp' not in new_path:
        new_paths_val.append([new_path, winner])


val_df = pd.DataFrame(new_paths_val)    
val_df[val_df.columns[1]]= val_df[val_df.columns[1]].astype(float)   
# val_df

In [5]:
# Create a dataloader class
class DatasetFaces_Rank(Dataset):
    
    def __init__(self, df, transform=None):
        self.data = df
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):

        img_name = self.data[self.data.columns[0]].iloc[index]
        # img_name = img_name.split('/')

   

        image = Image.open("/content/"+img_name).convert('RGB')
        # label = self.data[self.data.columns[1]].iloc[index]
        if self.transform is not None:
            image = self.transform(image)
        else:
            print("TRANSFORM FAILED")
        return image
  
# Create a dataloader class
class DatasetFaces_classify(Dataset):
    
    def __init__(self, df, transform=None):
        self.data = df
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):

        img_name = self.data[self.data.columns[0]].iloc[index]
        # img_name = img_name.split('/')

   

        image = Image.open("/content/"+img_name).convert('RGB')
        label = self.data[self.data.columns[1]].iloc[index]
        if self.transform is not None:
            image = self.transform(image)
        else:
            print("TRANSFORM FAILED")
        return image,label

In [6]:
# Create a transform for data prep
transform = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])


# Classification
train_dataset = DatasetFaces_classify(train_df, transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1,shuffle=False, num_workers=1)

test_dataset = DatasetFaces_classify(test_df, transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,shuffle=False, num_workers=1)


val_dataset = DatasetFaces_classify(val_df, transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1,shuffle=False, num_workers=1)


In [7]:
# file_name = train_data.iloc[0]
# print(file_name[0])
# img = Image.open('/content/'+ file_name[0])
# img

NameError: ignored

In [8]:
# Load model and create model
model = models.vgg16(pretrained=False)
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(25088, 4096),
    nn.Dropout(0.5),
    nn.Linear(4096, 4096),
    nn.Dropout(0.5),
    nn.Linear(4096, 1),
    nn.Sigmoid()
)

weights = torch.load('/content/drive/My Drive/Colab Notebooks/model weights/imagenet_vgg_3conv_best.pl')
model.load_state_dict(weights)
feature_extractor_first_layer = model.features[:30]
feature_extractor_second_layer = model.features[:28]
feature_extractor_third_layer = model.features[:26]

if torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("done")
print(model)
print(feature_extractor_first_layer)

using device: cuda
done
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=

In [9]:
train_features = []
test_features = []
val_features = []

with torch.no_grad():
        for data in train_loader:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs, label = data
            inputs = inputs.to(device)
            l29 = feature_extractor_first_layer(inputs)
            l27 = feature_extractor_second_layer(inputs)
            l25 = feature_extractor_third_layer(inputs)
            m = nn.MaxPool2d(16, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            train_features.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), label])
       
with torch.no_grad():
        for data in test_loader:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs, label = data
            inputs = inputs.to(device)
            l29 = feature_extractor_first_layer(inputs)
            l27 = feature_extractor_second_layer(inputs)
            l25 = feature_extractor_third_layer(inputs)
            m = nn.MaxPool2d(16, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            test_features.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), label])
            
with torch.no_grad():
        for data in val_loader:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs, label = data
            inputs = inputs.to(device)
            l29 = feature_extractor_first_layer(inputs)
            l27 = feature_extractor_second_layer(inputs)
            l25 = feature_extractor_third_layer(inputs)
            m = nn.MaxPool2d(16, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            val_features.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), label])


In [10]:
train_features = pd.DataFrame(train_features)
train_features.to_pickle('/content/drive/My Drive/Colab Notebooks/vgg16_imagenet_train.pl')
test_features = pd.DataFrame(test_features)
test_features.to_pickle('/content/drive/My Drive/Colab Notebooks/vgg16_imagenet_test.pl')
val_features = pd.DataFrame(val_features)
val_features.to_pickle('/content/drive/My Drive/Colab Notebooks/vgg16_imagenet_val.pl')

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


,0,1
0,"[[[[0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. 0. 0. ...",True
1,"[[[[0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. 0. 0. ...",True
2,"[[[[0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. 0. 0. ...",True
3,"[[[[0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. 0. 0. ...",True
4,"[[[[0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. 0. 0. ...",True
...,...,...
992,[[[[0. 0.41331547 1.2869496 1.4176927...,True
993,[[[[0. 0.41331547 1.2869496 1.4176927...,True
994,"[[[[0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. 0. 0. ...",True
995,"[[[[0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. 0. 0. ...",True
